<a href="https://colab.research.google.com/github/zhuchunlin1995/Deep-Learning/blob/master/image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#download the CIFAR10 dataset for image classification.
import torch, torchvision
import numpy as np
import random

transform = torchvision.transforms.Compose( [torchvision.transforms.ToTensor(),torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform)

#preprocess trainset.
dataset = []
#flat dataset
for data in trainset:
  tmp = []
  tmp.append(data[0].numpy().ravel())
  tmp.append(data[1])
  dataset.append(tmp)

#split dataset into training set and validation set
random.shuffle(dataset)
valiset = dataset[:5000]
trainset = dataset[5000:]

Files already downloaded and verified
3072


In [11]:
class NeuralNetwork:
  
  """we will initialize the number of neuron for every hidden layer as 2/3 size of the input layer,
    and assume the output layer will only contains only one neuron"""
  def __init__(self, layer_dimensions):
    
    #initialize weights and bias
    self.parameters = {}
    self.num_layers = len(layer_dimensions)
    for l in range (1, self.num_layers):
      eps = np.sqrt(2.0 / (layer_dimensions[l] + layer_dimensions[l - 1]))
      self.parameters["W" + str(l)] = np.random.randn(layer_dimensions[l], layer_dimensions[l - 1]) * eps
      self.parameters["b" + str(l)] = np.zeros((layer_dimensions[l], 1)) + 0.01
   
  def affineFoward(self, A, W, b):
    forward = np.dot(A, W) + b
    return forward
    
  def activationForward(self, A):
    # use relu function as activation function
    A = np.maximum(0, A)
    return A
  
  def costFunction(self, AL, y):
    #use softmax function to normalize AL first
    
    
  
  
  def affineBackward(self, dA_prev, cache):
  
  
    
    
    
    
if __name__ == "__main__":
  
  """we will initialize the number of neuron for every hidden layer as 2/3 size of the input layer,
    and assume the output layer will only contains only one neuron"""
  
  layer_dimensions = [3072, 2048, 1365, 1]
  nn = NeuralNetwork(layer_dimensions)
  print(nn.parameters["b1"])
      
    

[[0.01]
 [0.01]
 [0.01]
 ...
 [0.01]
 [0.01]
 [0.01]]
